In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator
from graph_helpers import *

%aimport GraphAPI
%aimport graph_helpers

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [2]:
gc = GraphCreator("Sonata cycle")
print("Number of Links to Search:", len(gc.next_links))
print(gc.primary_nodes)

Number of Links to Search: 147
{}


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [3]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect node. 

In [4]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [5]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,"Six Sonatas for Violin and Harpsichord, BWV 10...",Schubert's compositions for violin and piano,5
1,Sonata for two clarinets,Sonata for clarinet and bassoon,5
2,Schubert's compositions for violin and piano,"Six Sonatas for Violin and Harpsichord, BWV 10...",5
3,Cello Sonata (Poulenc),Sonata for clarinet and bassoon,5
4,Sonata for clarinet and bassoon,Cello Sonata (Poulenc),5


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [6]:
features_df = gc.get_features_df(rank=False)

In [7]:
features_df.head()

,node,degree,category_matches_with_source,in_edges,out_edges,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,primary_link
0,Ludwig van Beethoven,4869,0,4187.0,682.0,0.047560,0.119308,902.0,1.0,2.0,0
1,Music theory,2334,0,1649.0,685.0,0.005848,0.034134,300.0,1.0,2.0,0
2,Index of music articles,2037,0,114.0,1923.0,0.000784,0.002043,148.0,2.0,1.0,0
3,Movement (music),1545,0,1497.0,48.0,0.029523,0.025013,86.0,1.0,3.0,0
4,Sonata form,1021,2,713.0,308.0,0.103314,0.015250,362.0,2.0,2.0,0


## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [8]:
gc.rank_similarity()
gc.features_df[[
    "node", 
    "category_matches_with_source", 
    "primary_link", 
    "shortest_path_length_from_entry",
    "shortest_path_length_to_entry",
    "centrality",
    "similarity_rank", 
]].sort_values(["similarity_rank", "centrality"], ascending=False).head(50)
# gc.features_df[gc.features_df.category_matches_with_source == 1]

,node,category_matches_with_source,primary_link,shortest_path_length_from_entry,shortest_path_length_to_entry,centrality,similarity_rank
77,Sonata cycle,1,0,0.0,0.0,0.085738,inf
4,Sonata form,2,0,2.0,2.0,0.103314,1.0
44,Trio sonata,1,0,2.0,2.0,0.111817,0.5
45,Sonata rondo form,1,0,2.0,2.0,0.088484,0.5
7,Musical form,1,0,2.0,2.0,0.012638,0.5
41,Cyclic form,1,0,2.0,2.0,0.009529,0.5
11,Rondo,1,0,2.0,3.0,0.018420,0.4
23,Ternary form,1,0,2.0,3.0,0.013882,0.4
13,"Six Sonatas for Violin and Harpsichord, BWV 10...",0,0,1.0,1.0,0.175183,0.0
52,Violin Sonata (Poulenc),0,0,1.0,1.0,0.168977,0.0


In [ ]:
gc.intro_nodes

# Basic Plotting

In [ ]:
sns.pairplot(features_df)

In [ ]:
sns.heatmap(features_df.corr())

# Intro Node Values

In [ ]:
intro_nodes = gc.intro_nodes
intro_nodes_df = None
for node in intro_nodes:
    if df is None:
        intro_nodes_df = pd.DataFrame(features_df[features_df.node == node])
    else:
        intro_nodes_df = pd.concat([intro_nodes_df, pd.DataFrame(features_df[features_df.node == node])])
    
intro_nodes_df

In [ ]:
sns.pairplot(intro_nodes_df, hue="node")